# Code Setup

In [32]:
import numpy as np

from utils.import_puzzle_input import load_input
from utils.import_puzzle_input import split_puzzle_input

# Load the autoreload extension
%load_ext autoreload

# Set autoreload to automatically reload modules before executing code
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
puzzle_input = load_input(2023, 6)

loaded puzzle input


In [34]:
puzzle_input_split = split_puzzle_input(puzzle_input)

In [35]:
puzzle_input_split

['Time:        61     70     90     66',
 'Distance:   643   1184   1362   1041']

# Problem Setup

--- Day 6: Wait For It ---  
The ferry quickly brings you across Island Island. After asking around, you discover that there is indeed normally a large pile of sand somewhere near here, but you don't see anything besides lots of water and the   small island where the ferry has docked.
  
As you try to figure out what to do next, you notice a poster on a wall near the ferry dock. "Boat races! Open to the public! Grand prize is an all-expenses-paid trip to Desert Island!" That must be where the sand   comes from! Best of all, the boat races are starting in just a few minutes.
  
You manage to sign up as a competitor in the boat races just in time. The organizer explains that it's not really a traditional race - instead, you will get a fixed amount of time during which your boat has to travel   as far as it can, and you win if your boat goes the farthest.
  
As part of signing up, you get a sheet of paper (your puzzle input) that lists the time allowed for each race and also the best distance ever recorded in that race. To guarantee you win the grand prize, you need to   make sure you go farther in each race than the current record holder.
  
The organizer brings you over to the area where the boat races are held. The boats are much smaller than you expected - they're actually toy boats, each with a big button on top. Holding down the button charges the   boat, and releasing the button allows the boat to move. Boats move faster if their button was held longer, but time spent holding the button counts against the total race time. You can only hold the button at the start of the race, and boats don't move until the button is released.
  
For example:  
  
Time:      7  15   30  
Distance:  9  40  200  
This document describes three races:  
  
The first race lasts 7 milliseconds. The record distance in this race is 9 millimeters.  
The second race lasts 15 milliseconds. The record distance in this race is 40 millimeters.  
The third race lasts 30 milliseconds. The record distance in this race is 200 millimeters.  
Your toy boat has a starting speed of zero millimeters per millisecond. For each whole millisecond you spend at the beginning of the race holding down the button, the boat's speed increases by one millimeter per   millisecond.
  
So, because the first race lasts 7 milliseconds, you only have a few options:  
  
Don't hold the button at all (that is, hold it for 0 milliseconds) at the start of the race. The boat won't move; it will have traveled 0 millimeters by the end of the race.  
Hold the button for 1 millisecond at the start of the race. Then, the boat will travel at a speed of 1 millimeter per millisecond for 6 milliseconds, reaching a total distance traveled of 6 millimeters.  
Hold the button for 2 milliseconds, giving the boat a speed of 2 millimeters per millisecond. It will then get 5 milliseconds to move, reaching a total distance of 10 millimeters.  
Hold the button for 3 milliseconds. After its remaining 4 milliseconds of travel time, the boat will have gone 12 millimeters.  
Hold the button for 4 milliseconds. After its remaining 3 milliseconds of travel time, the boat will have gone 12 millimeters.  
Hold the button for 5 milliseconds, causing the boat to travel a total of 10 millimeters.  
Hold the button for 6 milliseconds, causing the boat to travel a total of 6 millimeters.  
Hold the button for 7 milliseconds. That's the entire duration of the race. You never let go of the button. The boat can't move until you let go of the button. Please make sure you let go of the button so the boat   gets to move. 0 millimeters.
Since the current record for this race is 9 millimeters, there are actually 4 different ways you could win: you could hold the button for 2, 3, 4, or 5 milliseconds at the start of the race.  
  
In the second race, you could hold the button for at least 4 milliseconds and at most 11 milliseconds and beat the record, a total of 8 different ways to win.  
  
In the third race, you could hold the button for at least 11 milliseconds and no more than 19 milliseconds and still beat the record, a total of 9 ways you could win.  
  
To see how much margin of error you have, determine the number of ways you can beat the record in each race; in this example, if you multiply these values together, you get 288 (4 * 8 * 9).  
  
Determine the number of ways you could beat the record in each race. What do you get if you multiply these numbers together?  

# General thoughts

This problem can largely be solved mathematically via second degree polynomial modelling.  
We observe that for boat race $i,$ a distance function $S$ might be had as
$$
S_{T_i}(\tau_i):=\tau_i(T_i - \tau_i) = -\tau_i^2 + T_i \cdot \tau_i
$$
With $T_i$ being the duration of the race, and $\tau_i$ being the time used for charging the boat.

With $m$ being the maximally had distance in previous races (the record) we thus need to estimate the cardinality of the set  
$$
\bar A_i:=\left\{\tau_i\in\mathbb{N}\,|\,S_{T_i}(\tau_i) \geq m_i \right\}
$$

We may acquire this set of positive integers (and thus measure its cardinality) by using the quadratic formula to find the real solutions of 
$$
S_{T_i}(\tau_i) - m_i = 0
$$
and checking the number of intermediate integers between the real solutions.  
This is a valid approach as we are dealing with a negative second degree coefficient causing $S$ to be larger for values of $\tau$ between any two real solutions.  

Thus, for the case of two real solutions, $\tau_i^{\ell}<\tau_i^{h},$ we might round up the lower real solution, and round down the higher real solution, and count the number of integers between these. 
$$
\bar A_i=\left\{k\in\mathbb{N}\,|\lceil \tau_i^{\ell} \rceil \leq k \leq \lfloor \tau_i^{h}\rfloor \right\}
$$
With this approach we need to check that we indeed preserve $ \lceil \tau_i^{\ell} \rceil \leq \lfloor \tau_i^{h}\rfloor.$

For only one real solution, we need to check whether it was itself an integer, and if no real roots exist, neither will there of course be integer solutions.

Solutions to this problem might be had via the quadratic formula:
$$
\hat{\tau_i}=\frac{-T_i\pm\sqrt{T_i^2-4m_i}}{-2}
$$

# Solutions

## Solution 1:

### Explanation of method

1. Retrieve the data for the duration, and the record of each game.  
2. Make a function to solve the quadratic    
2.1. This function should return a tuple of numbers with the real solutions (if any) to the given race problem  
3. Create a function to find the number of integers between two floats  
3.1. This function should take in the tuple from above.    
3.2. If the tuple contains no elements, `return 0`  
3.3. If the tuple contains one element, check if it is an integer. If it is an integer, `return 1`, else `return 0`  
3.4. If the tuple has two elements, round the upper one down and the lower one up.   
3.5. Check whether there is no integer between the two real solutions, if so `return 0`  
3.6. Check whether there is one integer between the two real solutions, including one (and only one) of the real solutions being an integer itself, if so `return 1`  
3.7. Check whether there is multiple integers between the real solutions, including the case for which both the real solutions are integers.   
4. Run over the games and take the product of the number of solutions to each  

#### Remarks

Whether a number is an integer is in this solution checked via the `float.is_integer()` method.  
It's precision is investigated in the experimenting section at the bottom.

### Implementation

#### Part 1

In [36]:
def get_data(puzzle_input_split: list[str]) -> tuple:
    durations = [int(x) for x in puzzle_input_split[0].split(" ") if x != "" and x != "Time:"]
    records = [int(x) for x in puzzle_input_split[1].split(" ") if x != "" and x != "Distance:"]

    return durations, records

#### Part 2

In [ ]:
def solve_quadratic_real(T: int, m: int) -> tuple:
    discriminant = T**2 - 4 * m

    if discriminant > 0:
        # Two real solutions
        l_sol = (-T + np.sqrt(T**2 - 4 * m))/(-2)
        u_sol = (-T - np.sqrt(T**2 - 4 * m))/(-2)
        return (l_sol, u_sol)
    elif discriminant == 0:
        sol = (-T - np.sqrt(T**2 - 4 * m))/(-2)
        return tuple(sol)
    else:
        return None

#### Part 3

In [38]:
def find_number_of_integers(sol_tuple: tuple[float]) -> int:
    if not sol_tuple:
        return 0
    elif len(sol_tuple) == 1:
        if sol_tuple[0].is_integer():
            return 1
        else:
            return 0
    else:
        # Assuming length 2
        l_sol = sol_tuple[0]
        u_sol = sol_tuple[1]

        # Taking the ceiling of the lower solution
        # and the floor of the upper
        l_sol_ceil = np.ceil(l_sol)
        u_sol_floor = np.floor(u_sol)

        if u_sol_floor < l_sol_ceil:
            # solutions are between the same two integers
            # with no integer between them
            return 0
        elif l_sol_ceil == u_sol_floor:
            # The solutions have exactly one integer between them
            return 1
        else:
            # Solutions have multiple integers between them
            
            # Note np.ceil and np.floor are implemented with a rounding domain
            # that leaves integers themselves unrounded
            # Thus we do not need to be concerned about the case where 
            # u_sol is an integer for which u_sol = l_sol + 1
            return u_sol_floor - l_sol_ceil + 1

#### Part 4

In [39]:
def solve_puzzle(puzzle_input_split: list[str]) -> int:
    durations, records = get_data(puzzle_input_split)

    number_of_solutions = [find_number_of_integers(solve_quadratic_real(duration, record)) for duration, record in zip(durations, records)]

    return np.prod(number_of_solutions)

In [ ]:
solve_puzzle(puzzle_input_split)

### Checking runtime

#### Benchmarking

#### Runtime profiling

## Solution 2:

# Experimental

## Retrivning the data

In [5]:
puzzle_input_split

['Time:        61     70     90     66',
 'Distance:   643   1184   1362   1041']

In [6]:
puzzle_input_split[0]

'Time:        61     70     90     66'

In [ ]:
puzzle_input_split[0].split(" ")

['Time:',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '61',
 '',
 '',
 '',
 '',
 '70',
 '',
 '',
 '',
 '',
 '90',
 '',
 '',
 '',
 '',
 '66']

In [9]:
test_space_split_time = puzzle_input_split[0].split(" ")
test_space_split_time

['Time:',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '61',
 '',
 '',
 '',
 '',
 '70',
 '',
 '',
 '',
 '',
 '90',
 '',
 '',
 '',
 '',
 '66']

In [15]:
[int(x) for x in test_space_split_time if x != '' and x != "Time:"]

[61, 70, 90, 66]

In [16]:
puzzle_input_split[1].split(" ")

['Distance:', '', '', '643', '', '', '1184', '', '', '1362', '', '', '1041']

In [17]:
[int(x) for x in puzzle_input_split[1].split(" ") if x != "" and x != "Distance:"]

[643, 1184, 1362, 1041]

## Numerical precision in python

In [ ]:
def get_epsilon():
    while_counter = 0
    max_iter = 1e4
    num = 10.0
    add_value = 1e-6
    num_p_av = num + add_value
    while not num_p_av.is_integer() and while_counter <= max_iter:
        add_value /= 2.0
        num_p_av = num + add_value
        while_counter += 1
    
    if while_counter > max_iter:
        return None
    else:
        return add_value

In [ ]:
safe_tol = get_epsilon()
safe_tol

4.656612873077392e-16

In [ ]:
np.ceil(10.0 + safe_tol)

np.float64(10.0)

In [ ]:
np.floor(11.0 - safe_tol)

np.float64(11.0)

In [ ]:
np.ceil(10.0 + 2 * safe_tol)

np.float64(11.0)

In [ ]:
num = 10.0
tol = 1e-6
num_p_tol = num + tol

In [ ]:
num_p_tol.is_integer()

False

In [34]:
test_list = [2,3,4]

In [35]:
np.prod(test_list)

np.int64(24)